# Plot meshing and boundaries on the Earth

In [ ]:
import numpy as np     
import plotly.graph_objects as go
import IPython
from numpy import pi, cos, sin
import gstlearn as gl 

Functions to convert long lat into 3d coordinates :

The first step is to create a meshing and a white skin on this meshing.

In [ ]:
gl.variety_define(1)

mesh = gl.MeshSpherical()
mesh.reset(None,None,triswitch = "-r4",verbose=False)

X = mesh.getCoordinates(0)
Y = mesh.getCoordinates(1)

bl = np.array(gl.util_convert_longlat(X,Y,dilate=0.99))
             

meshes = np.array(mesh.getMeshes()).reshape([mesh.getNMeshes(),3])-1

mesh.getMeshes()

blank = go.Mesh3d(
        x=bl[0,:],
        y=bl[1,:],
        z=bl[2,:],
        colorbar_title='z',
        # Intensity of each vertex, which will be interpolated and color-coded
        intensity=np.zeros(shape=bl[0,:].shape[0]),
        # i, j and k give the vertices of triangles
        # here we represent the 4 triangles of the tetrahedron surface
        i=meshes[:,0],
        j=meshes[:,1],
        k=meshes[:,2],
        name='y',
        showscale=False
    )



Then we build the vectors of coordinates of each triangle. We can use the gstlearn function **AMesh::getEmbeddedCoordinatesPerMesh**

In [ ]:
x = list()
y = list()
z = list()
for i in range(mesh.getNMeshes()):
    a = np.array(mesh.getEmbeddedCoordinatesPerMesh(i))
    x.extend(a[:,0].tolist() + [None])
    y.extend(a[:,1].tolist() + [None])
    z.extend(a[:,2].tolist() + [None])

x = np.array(x)
y = np.array(y)
z = np.array(z)

Now we plot the meshing on the white skin

In [ ]:
meshing=dict(type='scatter3d',
               x=x,
               y=y,
               z=z,
               mode='lines',
               line=dict(color='black', width=1)
              )

fig = go.Figure(data = [blank,meshing]
               )
fig.show()

## Draw from a polygon of gstlearn

In [ ]:

def d2r(x):
    return x * np.pi / 180.

def mapping_map_to_sphere(lon, lat, radius=6371.):
    #this function maps the points of coords (lon, lat) to points onto the  sphere of radius radius
    
    lon=np.array(lon, dtype=np.float64)
    lat=np.array(lat, dtype=np.float64)
    lon=d2r(lon)
    lat=d2r(lat)
    xs=radius*cos(lon)*cos(lat)
    ys=radius*sin(lon)*cos(lat)
    zs=radius*sin(lat)
    return xs, ys, zs


In [ ]:
poly = gl.Polygons.createFromNF("../../../doc/data/boundaries/world.poly")

In [ ]:
X = list()
Y = list()

for i in range(poly.getPolySetNumber()):
    a = list(poly.getX(i))
    b = list(poly.getY(i))
    X.extend(a + [None])
    Y.extend(b + [None])
    
X = np.array(X)
Y = np.array(Y)

xs,ys,zs = mapping_map_to_sphere(np.array(X),np.array(Y),6371*1.0)

In [ ]:
boundaries=dict(type='scatter3d',
               x=xs,
               y=ys,
               z=zs,
               mode='lines',
               line=dict(color='black', width=1)
              )

fig = go.Figure(data = [blank,boundaries])
fig.show()